# Data Acquision

In [11]:
import yfinance as yf
from fredapi import Fred
from config import get_fredapi_key

import numpy as np
import pandas as pd

# define fred api (using my personal api key)
fred = Fred(api_key = get_fredapi_key())

In [36]:
# define dataframe
df = pd.DataFrame()

## S&P500

https://finance.yahoo.com/quote/%5EGSPC/

In [37]:
# download data as ticker "^GSPC" and convert to monthly data
sp500 = yf.download(['^GSPC']).resample('M').last().reset_index()

[*********************100%***********************]  1 of 1 completed


In [38]:
# add "Date" (format: yyyy-mm) column to "df"
df['Date'] = sp500['Date'].dt.strftime('%Y-%m')

# create new columns: "Year" and "Month" and add them to "df"
df['Year'] = sp500['Date'].dt.year
df['Month'] = sp500['Date'].dt.month

# add S&P500 data
df['SP500'] = sp500['Close']

# calculate % change of YoY
df['SP500_%YoY'] = df['SP500'].pct_change(12)

# calculate % change of MoM
df['SP500_%MoM'] = df['SP500'].pct_change()

df.head()

,Date,Year,Month,SP500,SP500_%YoY,SP500_%MoM
0,1927-12,1927,12,17.660000,NaN,NaN
1,1928-01,1928,1,17.570000,NaN,-0.005096
2,1928-02,1928,2,17.260000,NaN,-0.017644
3,1928-03,1928,3,19.280001,NaN,0.117034
4,1928-04,1928,4,19.750000,NaN,0.024378


## Economic data from Fred

- MY10Y: https://fred.stlouisfed.org/series/DGS10
- CPI: https://fred.stlouisfed.org/series/CPIAUCSL
- M2SL: https://fred.stlouisfed.org/series/M2SL
- CSENT: https://fred.stlouisfed.org/series/UMCSENT
- IPM: https://fred.stlouisfed.org/series/IPMAN
- UNEMP: https://fred.stlouisfed.org/series/UNRATE

In [44]:
# list of ids and column names of monthly data
ids = ['DGS10', 'CPIAUCSL', 'M2SL', 'UMCSENT', 'IPMAN', 'UNRATE']
cols = ['MY10Y', 'CPI', 'M2', 'CSENT', 'IPM', 'UNEMP']

# travering all pairs of ids and cols
for id, col in zip(ids, cols):
    # get the original data
    original = fred.get_series(id)
    # convert current index to column & change monthly data
    df_merge = original.resample('M').last().reset_index()
    # change column names
    df_merge.columns = ['Date', col]
    # update format of 'Date' column
    df_merge['Date'] = df_merge['Date'].dt.strftime('%Y-%m')

    # for unemployment rate, nothing to do
    if col == 'UNEMP':
        continue
    # otherwise, add new columns
    else:
        # add derived data (%YoY & %MoM)
        df_merge[f'{col}_%YoY'] = df_merge[col].pct_change(12)
        df_merge[f'{col}_%MoM'] = df_merge[col].pct_change()

    # merge two data frames
    df = pd.merge(df, df_merge, on='Date', how='left') # type: ignore


df.head() # type: ignore

,Date,Year,Month,SP500,SP500_%YoY,SP500_%MoM,MY10Y,MY10Y_%YoY,MY10Y_%MoM,CPI,...,M2,M2_%YoY,M2_%MoM,CSENT,CSENT_%YoY,CSENT_%MoM,IPM,IPM_%YoY,IPM_%MoM,UNEMP
0,1927-12,1927,12,17.660000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1928-01,1928,1,17.570000,NaN,-0.005096,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1928-02,1928,2,17.260000,NaN,-0.017644,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1928-03,1928,3,19.280001,NaN,0.117034,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1928-04,1928,4,19.750000,NaN,0.024378,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Data Clearning

In [66]:
# dropping rows based on any missing data
df_clean = df.dropna()

# groupby dataframe based on "Year" and check the count
df_clean.groupby('Year').count()[['Date']]

,Date
Year,
1973,4
1974,4
1975,4
1976,4
1977,4
1978,12
1979,12
1980,12
1981,12


In [65]:
# Based on the observation, the data between 1973 and 1977 has missing in some months.
# Thus, removing data between 1973 and 1977.
df_clean = df_clean[df_clean['Year'] >= 1978].reset_index(drop=True)
df_clean.head()

,Date,Year,Month,SP500,SP500_%YoY,SP500_%MoM,MY10Y,MY10Y_%YoY,MY10Y_%MoM,CPI,...,M2,M2_%YoY,M2_%MoM,CSENT,CSENT_%YoY,CSENT_%MoM,IPM,IPM_%YoY,IPM_%MoM,UNEMP
0,1978-01,1978,1,89.250000,-0.125257,-0.061514,7.94,0.072973,0.020566,62.7,...,1279.7,0.098266,0.007400,83.7,-0.037931,-0.008294,43.7471,0.068292,-0.015463,6.4
1,1978-02,1978,2,87.040001,-0.128030,-0.024762,8.04,0.079195,0.012594,63.0,...,1285.5,0.091627,0.004532,84.3,-0.032147,0.007168,43.9139,0.052995,0.003813,6.3
2,1978-03,1978,3,89.209999,-0.093579,0.024931,8.15,0.098383,0.013682,63.4,...,1292.2,0.087253,0.005212,78.8,-0.095293,-0.065243,44.6899,0.056017,0.017671,6.3
3,1978-04,1978,4,96.830002,-0.016355,0.085416,8.24,0.106040,0.011043,63.9,...,1300.4,0.084028,0.006346,81.6,-0.063146,0.035533,45.3905,0.061632,0.015677,6.1
4,1978-05,1978,5,97.239998,0.011652,0.004234,8.42,0.140921,0.021845,64.5,...,1310.5,0.083954,0.007767,82.9,-0.080931,0.015931,45.6235,0.058535,0.005133,6.0
